# C2Cシェアサイクルシステム

## 問題設定
個人間で自転車をシェアリングし，可能な限り乗り捨て可能とするシステムについて考える．利用可能な自転車は複数台存在し，それぞれが任意のポイントに駐輪されているとする．本システムのユーザーのサービス利用リクエストを時間幅Tの間隔で分割し，それぞれのリクエストに合わせた最適な自転車を割り当てる．

サービス利用後に任意の場所に駐輪された自転車を定位置に再配置するコストを最小化し，　より多くのユーザーに自転車を割り当てることを目的として，最適化な自転車をユーザーに割り当てる．

なお，ここではMVPとして，時間幅T=1(分)とした小規模のデータを利用することとし，最終アウトプットとしては「どのユーザーにどの自転車が割り当てられたか」と「移動後の自転車の配置状況・利用可能状況」を期待する．

[fig]

### データ
データとしては，以下のような情報を必要とする．
- 定数
 - $T$：ユーザーリクエストを区切る時間幅
- 変数
 - $J$：ユーザーリクエストデータ(データフレーム？)
 - $B$：利用可能な自転車データ(データフレーム？)

 etc

In [ ]:
# ライブラリのインストール
!pip install ortools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.2/309.2 kB 19.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires protobuf<5,>=3.20, but you have protobuf 5.27.1 which is incompatible.
google-ai-generativelanguage 0.6.4 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have proto

In [ ]:
import branca.colormap as cm
import folium
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from geopy.distance import geodesic
from ortools.linear_solver import pywraplp
from pandas import DataFrame

In [ ]:
# データの準備

'''locationID検索用CSV'''
df_locations = pd.read_csv('/content/taxi_zone_lookup_with_coordinates.csv')
# df_locations.set_index("LocationID", inplace=True)
print(df_locations.head())


# LocationIDから経度と緯度をタプルで返す関数
def get_coordinates_by_location_id(location_id):
    row = df_locations[df_locations['LocationID'] == location_id]
    if not row.empty:
        return (row.iloc[0]['Latitude'], row.iloc[0]['Longitude'])
    else:
        return None

   LocationID        Borough                     Zone service_zone   Latitude  \
0           1            EWR           Newark Airport          EWR  40.689531   
1           2         Queens              Jamaica Bay    Boro Zone  40.603994   
2           3          Bronx  Allerton/Pelham Gardens    Boro Zone  40.865229   
3           4      Manhattan            Alphabet City  Yellow Zone  40.725102   
4           5  Staten Island            Arden Heights    Boro Zone  40.563700   

   Longitude  
0 -74.174462  
1 -73.835412  
2 -73.842739  
3 -73.979583  
4 -74.191603  


In [ ]:
'''自転車の集合'''
# ランダムシードを設定して、ランダムに10個選択
np.random.seed(42)
random_sample = df_locations.sample(n=10)

# Bike IDを設定
random_sample['Bike ID'] = range(10)

# 緯度と経度をホームポジションとカレントポジションに設定
random_sample['Home Position'] = list(zip(random_sample['Latitude'], random_sample['Longitude']))
random_sample['Current Location'] = random_sample['Home Position']

# 結果のデータフレームを整形
B = random_sample[['Bike ID', 'Home Position', 'Current Location']]
B.set_index("Bike ID", inplace=True)

# データの中身を確認
B

,Home Position,Current Location
Bike ID,,
0,"(40.67677, -73.8437461)","(40.67677, -73.8437461)"
1,"(40.8241451, -73.9500618)","(40.8241451, -73.9500618)"
2,"(40.6907711, -73.9766245)","(40.6907711, -73.9766245)"
3,"(40.68562615, -73.98417065807277)","(40.68562615, -73.98417065807277)"
4,"(40.67592055, -73.78496487588887)","(40.67592055, -73.78496487588887)"
5,"(40.76883397436158, -73.95193997045698)","(40.76883397436158, -73.95193997045698)"
6,"(40.70533183168504, -73.95019177498656)","(40.70533183168504, -73.95019177498656)"
7,"(40.8473226, -73.7865218)","(40.8473226, -73.7865218)"
8,"(40.750201, -73.993104)","(40.750201, -73.993104)"


In [ ]:
'''ユーザーリクエストの集合'''
# ParquetファイルのURL
url = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet'

# Parquetファイルを読み込む
df = pd.read_parquet(url)

# 指定されたカラムのみを含むデータフレームを取得
df_requests = df[['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'PULocationID', 'DOLocationID']]

# データのフィルタリング
# 2023年1月1日以前のデータを削除
df_requests = df_requests[df_requests['tpep_pickup_datetime'] >= '2023-01-01']
# 2023年2月1日以降のデータを削除
df_requests = df_requests[df_requests['tpep_pickup_datetime'] <= '2023-01-31']

# ピックアップタイムの昇順で並び替え
df_requests = df_requests.sort_values(by='tpep_pickup_datetime')

# インデックスをリセット
df_requests = df_requests.reset_index(drop=True)

# フィルタリングされたデータの先頭を表示
print(df_requests.head())

# データフレームの情報を表示
print(df_requests.info())

  tpep_pickup_datetime tpep_dropoff_datetime  PULocationID  DOLocationID
0  2023-01-01 00:00:00   2023-01-01 00:08:00            42            41
1  2023-01-01 00:00:05   2023-01-01 00:26:27           249           186
2  2023-01-01 00:00:06   2023-01-01 00:05:44           125            68
3  2023-01-01 00:00:08   2023-01-01 00:11:24            42           244
4  2023-01-01 00:00:09   2023-01-01 00:15:10            79           231
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2966346 entries, 0 to 2966345
Data columns (total 4 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   tpep_pickup_datetime   datetime64[us]
 1   tpep_dropoff_datetime  datetime64[us]
 2   PULocationID           int64         
 3   DOLocationID           int64         
dtypes: datetime64[us](2), int64(2)
memory usage: 90.5 MB
None


In [ ]:
# モデリングするためにユーザーリクエストデータを整形する

# tpep_pickup_datetimeをdatetime型に変換
df_requests['tpep_pickup_datetime'] = pd.to_datetime(df_requests['tpep_pickup_datetime'])
df_requests['tpep_dropoff_datetime'] = pd.to_datetime(df_requests['tpep_dropoff_datetime'])

# 一か月分のデータを一分ごとに分割
# start_time = df_requests['tpep_pickup_datetime'].min()
# end_time = df_requests['tpep_pickup_datetime'].max()

# 最初の1分のデータを抽出
start_time = df_requests['tpep_pickup_datetime'].min()
end_time = start_time + pd.Timedelta(minutes=1)

# まずは最初の1分間のリクエストをユーザーリクエストの集合として扱う
J = df_requests[(df_requests['tpep_pickup_datetime'] >= start_time) & (df_requests['tpep_pickup_datetime'] < end_time)]

# type(J)
J

,tpep_pickup_datetime,tpep_dropoff_datetime,PULocationID,DOLocationID
0,2023-01-01 00:00:00,2023-01-01 00:08:00,42,41
1,2023-01-01 00:00:05,2023-01-01 00:26:27,249,186
2,2023-01-01 00:00:06,2023-01-01 00:05:44,125,68
3,2023-01-01 00:00:08,2023-01-01 00:11:24,42,244
4,2023-01-01 00:00:09,2023-01-01 00:15:10,79,231
5,2023-01-01 00:00:13,2023-01-01 00:12:52,132,7
6,2023-01-01 00:00:18,2023-01-01 00:09:34,238,262
7,2023-01-01 00:00:22,2023-01-01 00:26:23,136,233
8,2023-01-01 00:00:35,2023-01-01 00:25:12,132,17
9,2023-01-01 00:00:47,2023-01-01 00:04:32,79,107


In [ ]:
'''ユーザーリクエストJに対して移動後の自転車Bの位置関係を表す距離行列を返す関数'''
def generate_after_trip_distances(
    df_bikes: DataFrame,
    df_requests: DataFrame,
) -> np.ndarray:

    # 自転車とリクエストの数
    num_bikes = len(df_bikes)
    num_requests = len(df_requests)

    # 移動後の距離行列 d を作成 (d[b, j] が利用者 j が移動した後の自転車 b とその定位置までの距離)
    # 距離行列を初期化
    after_trip_distances = np.zeros((num_bikes, num_requests))
    for b in range(num_bikes):
        home_position = df_bikes.loc[b, 'Home Position']
        for j in range(num_requests):
            request_destination_id = df_requests.loc[j, 'DOLocationID']
            request_destination = get_coordinates_by_location_id(request_destination_id)
            after_trip_distances[b, j] = geodesic(
                home_position, request_destination
            ).m  # 単位はメートル

    return after_trip_distances

In [ ]:
# デバッグ
# ユーザーリクエストJに対して移動された自転車Bにおける、自転車の定位置との距離行列
distances = generate_after_trip_distances(B, J)
print(distances)

[[16324.6965866  15025.69004275 15406.86232578 19990.18350913
  14907.19068104 12856.90047253 14196.22025405 13245.07257533
  10638.86535405 13535.28056899 16373.93306076 11340.10038402]
 [ 2880.37709413  8979.25090277  9655.74897413  2107.14841695
  12971.40478868  6025.30017515  5006.95320151  8415.52519669
  11719.56250095 10252.2746341   2851.99880036 14979.9612261 ]
 [12110.43982081  6744.87746968  6527.93089092 17013.87720116
   4186.64777432  9835.22134701 10204.78938704  6572.77049439
   3524.74055425  5008.93846511 12134.94413876     0.        ]
 [12791.84800114  7210.55299741  6909.50261733 17700.2961098
   4220.59321199 10619.22053993 10936.19640361  7234.50307073
   4339.58949175  5540.51132054 12814.43064257   856.33837516]
 [19620.20207123 19425.83248026 19896.42120853 22539.76440839
  19723.18936374 16265.6081012  17680.1909753  17488.56913273
  15290.03178085 18081.58266884 19671.64681594 16285.95097795]
 [ 3269.84196753  4045.16769894  4861.29000959  8122.40191257
   7

In [ ]:
'''ユーザーの位置と自転車の位置をプロットする関数'''
def plot_users_and_bikes(
    user_locations: np.ndarray,
    bike_locations: np.ndarray,
    latitude_range: tuple[float, float],  # 描画範囲 (緯度)
    longitude_range: tuple[float, float],  # 描画範囲 (経度)
):
    m = folium.Map(
        [sum(latitude_range) / 2, sum(longitude_range) / 2],
        tiles="OpenStreetMap",
        zoom_start=11,
    )

    for latitude, longitude in user_locations:
        folium.Marker(
            location=(latitude, longitude),
            icon=folium.Icon(icon="user", prefix="fa", color="orange"),
        ).add_to(m)

    for latitude, longitude in bike_locations:
        folium.Marker(
            location=(latitude, longitude),
            icon=folium.Icon(icon="bicycle", prefix="fa", color="green"),
        ).add_to(m)

    return m

In [ ]:
# latitudeカラムとlongitudeカラムの最大値と最小値を取得
latitude_max = df_locations['Latitude'].max()
latitude_min = df_locations['Latitude'].min()
longitude_max = df_locations['Longitude'].max()
longitude_min = df_locations['Longitude'].min()

# 結果を表示
print(f"Latitude: max = {latitude_max}, min = {latitude_min}")
print(f"Longitude: max = {longitude_max}, min = {longitude_min}")

Latitude: max = 40.908178, min = 40.52825085411132
Longitude: max = -73.7115199, min = -74.23462644152289


In [ ]:
# NYC
latitude_range = (latitude_min - 0.1, latitude_max + 0.1)
longitude_range = (longitude_min - 0.1, longitude_max + 0.1)
print(latitude_range)
print(longitude_range)

# 自転車の現在地の配列
current_locations = B['Current Location'].values
print(type(current_locations))
print(current_locations)

# ユーザーの現在地の配列
request_origin_ids = J['PULocationID']
request_origins = []
for origin_id in request_origin_ids:
    request_origins.append(get_coordinates_by_location_id(origin_id))
request_origins = np.array(request_origins)
print(type(request_origins))
print(request_origins)

(40.42825085411132, 41.008178)
(-74.33462644152289, -73.6115199)
<class 'numpy.ndarray'>
[(40.67677, -73.8437461) (40.8241451, -73.9500618)
 (40.6907711, -73.9766245) (40.68562615, -73.98417065807277)
 (40.67592055, -73.78496487588887) (40.76883397436158, -73.95193997045698)
 (40.70533183168504, -73.95019177498656) (40.8473226, -73.7865218)
 (40.750201, -73.993104) (40.8126008, -73.8840247)]
<class 'numpy.ndarray'>
[[ 40.7985612  -73.9530142 ]
 [ 40.7341857  -74.00558   ]
 [ 40.7268035  -74.0079833 ]
 [ 40.7985612  -73.9530142 ]
 [ 40.7292688  -73.9873613 ]
 [ 40.6429479  -73.77937337]
 [ 40.7965512  -73.96843153]
 [ 40.8725702  -73.9026619 ]
 [ 40.6429479  -73.77937337]
 [ 40.7292688  -73.9873613 ]
 [ 40.7598219  -73.9724708 ]
 [ 40.7757145  -73.87336399]]


In [ ]:
plot_users_and_bikes(request_origins, current_locations, latitude_range, longitude_range)

In [ ]:
# 問題の正規化
average = distances.mean()
print(f"average: {average}")
std = distances.std()
print(f"std: {std}")
distances: np.ndarray = (distances - average) / std
print(f"distances: {distances}")

average: -1.0177044392397268e-17
std: 1.0
distances: [[ 1.02174276  0.80564109  0.8690526   1.6315302   0.78592763  0.44484301
   0.66765117  0.50941901  0.07585249  0.55769778  1.0299337   0.19250938]
 [-1.21484333 -0.20023963 -0.08769796 -1.34347703  0.46389186 -0.69165648
  -0.86106785 -0.29402058  0.25563639  0.01153957 -1.21956432  0.79803368]
 [ 0.32066242 -0.57194818 -0.60803924  1.13639429 -0.9975332  -0.05784108
   0.00363996 -0.60057976 -1.10764755 -0.86073759  0.32473894 -1.69402053]
 [ 0.43402094 -0.49447878 -0.54456127  1.25058638 -0.99188606  0.07258439
   0.12531624 -0.49049446 -0.97208994 -0.77230556  0.43777777 -1.55156077]
 [ 1.56998038  1.53764522  1.61593198  2.05567641  1.58711327  1.01191285
   1.24724171  1.21536366  0.84961676  1.31401691  1.57853869  1.01529707]
 [-1.15005227 -1.02106971 -0.88530026 -0.34278433 -0.42146797 -1.38400631
  -1.47211506 -1.2736895  -0.76593752 -0.93069151 -1.14462838 -0.21075133]
 [ 0.023337   -0.66892816 -0.64750886  0.82358881 -0.

In [ ]:
# OR-Toolsのソルバーを作成
solver = pywraplp.Solver.CreateSolver('SCIP')

# 変数の定義
x = []
for b in range(B.shape[0]):
    x.append([])
    for j in range(J.shape[0]):
        x[b].append(solver.BoolVar(f'x[{b},{j}]'))
print(x)

alpha = 1.0

# 目的関数の定義
# 第一項: ユーザーの移動後の自転車の現在地と定位置との距離を短くする
distance_objective = solver.Sum(distances[b][j] * x[b][j] for b in range(B.shape[0]) for j in range(J.shape[0]))
# 第二項: より多くのユーザーに自転車を割り当てる
sum_x = solver.Sum(x[b][j] for b in range(B.shape[0]) for j in range(J.shape[0]))

objective = distance_objective - alpha * sum_x
solver.Minimize(objective)

# 制約条件の定義

# 各ユーザーは1台の自転車にしか割り当てられない
for b in range(B.shape[0]):
    solver.Add(solver.Sum(x[b][j] for j in range(J.shape[0])) <= 1)

# 各自転車は１人のユーザーにしか割り当てられない
for j in range(J.shape[0]):
    solver.Add(solver.Sum(x[b][j] for b in range(B.shape[0])) <= 1)

# ソルバーを実行
status = solver.Solve()
print(status)

if status == pywraplp.Solver.OPTIMAL:
    print('解が見つかりました:')
    bike_assignment = []
    for b in range(B.shape[0]):
        for j in range(J.shape[0]):
            if x[b][j].solution_value() == 1:
                bike_assignment.append((b, j))
                print(f"利用者 {j}: 自転車 {b}")
else:
    raise RuntimeError("No feasible solution was found.")

[[x[0,0], x[0,1], x[0,2], x[0,3], x[0,4], x[0,5], x[0,6], x[0,7], x[0,8], x[0,9], x[0,10], x[0,11]], [x[1,0], x[1,1], x[1,2], x[1,3], x[1,4], x[1,5], x[1,6], x[1,7], x[1,8], x[1,9], x[1,10], x[1,11]], [x[2,0], x[2,1], x[2,2], x[2,3], x[2,4], x[2,5], x[2,6], x[2,7], x[2,8], x[2,9], x[2,10], x[2,11]], [x[3,0], x[3,1], x[3,2], x[3,3], x[3,4], x[3,5], x[3,6], x[3,7], x[3,8], x[3,9], x[3,10], x[3,11]], [x[4,0], x[4,1], x[4,2], x[4,3], x[4,4], x[4,5], x[4,6], x[4,7], x[4,8], x[4,9], x[4,10], x[4,11]], [x[5,0], x[5,1], x[5,2], x[5,3], x[5,4], x[5,5], x[5,6], x[5,7], x[5,8], x[5,9], x[5,10], x[5,11]], [x[6,0], x[6,1], x[6,2], x[6,3], x[6,4], x[6,5], x[6,6], x[6,7], x[6,8], x[6,9], x[6,10], x[6,11]], [x[7,0], x[7,1], x[7,2], x[7,3], x[7,4], x[7,5], x[7,6], x[7,7], x[7,8], x[7,9], x[7,10], x[7,11]], [x[8,0], x[8,1], x[8,2], x[8,3], x[8,4], x[8,5], x[8,6], x[8,7], x[8,8], x[8,9], x[8,10], x[8,11]], [x[9,0], x[9,1], x[9,2], x[9,3], x[9,4], x[9,5], x[9,6], x[9,7], x[9,8], x[9,9], x[9,10], x[9,11]]]

In [ ]:
print(bike_assignment)

[(0, 5), (1, 10), (2, 11), (3, 4), (5, 6), (6, 8), (7, 3), (8, 1), (9, 0)]


In [ ]:
# debug
# for i in range(10):
#   for j in range(12):
#     print(x[i][j].solution_value())

In [ ]:
'''自転車とユーザーを，割り当てられた自転車ごとに異なる色で塗り分けてプロットする関数'''
def plot_result(
    bike_assignment: list[tuple[int, int]],
    user_locations: np.ndarray,
    bike_locations: np.ndarray,
    latitude_range: tuple[float, float],  # 描画範囲 (緯度)
    longitude_range: tuple[float, float],  # 描画範囲 (経度)
):
    # マップを用意
    m = folium.Map(
        [sum(latitude_range) / 2, sum(longitude_range) / 2],
        tiles="OpenStreetMap",
        zoom_start=11,
    )

    # 色の用意
    colormap = cm.linear.Set1_09.scale(0, len(bike_locations)).to_step(len(bike_locations))  # type: ignore

    # 車のプロット (k 番目の自転車を色 k で塗る)
    for bike_index, (latitude, longitude) in enumerate(bike_locations):
        folium.Marker(
            location=(latitude, longitude),
            popup=f"bike {bike_index}",
            icon=folium.Icon(
                icon="bicycle", prefix="fa", color="white", icon_color=colormap(bike_index)
            ),
        ).add_to(m)

    # 利用者のプロット (自転車 k に乗るユーザーを色 k で塗る)
    for bike_index, user_index in bike_assignment:
        latitude, longitude = user_locations[user_index]
        folium.Marker(
            location=(latitude, longitude),
            popup=f"bike {bike_index}",
            icon=folium.Icon(
                icon="user",
                prefix="fa",
                color="white",
                icon_color=colormap(bike_index),
            ),
        ).add_to(m)

    return m

In [ ]:
plot_result(bike_assignment, request_origins, current_locations, latitude_range, longitude_range)



---

